In [5]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F 
import random
%matplotlib inline 

In [6]:
words =  open("names.txt").read().splitlines()

In [7]:
len(words)

32033

In [8]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}

In [9]:
block_size = 3

def build_dataset(word):
    X, Y = [], []

    for w in word:
        context = [0] * block_size

        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [18]:
class Linear:

    def __init__(self, fan_in, fan_out, bias = True):
        self.weight = torch.randn((fan_in,fan_out)) * (fan_in ** 0.5)
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + [self.bias] if self.bias is not None else []

class Tanh():

    def __call__(self, x):
        return torch.tanh(x)
    
    def parameters(self):
        return []
    
class BatchNorm1d:

    def __init__(self, dim, eps = 1e-05, momentum = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.gamma = torch.ones(dim) 
        self.beta = torch.zeros(dim)
        self.training = True
        self.running_mean = torch.zeros(dim)
        self.running_std = torch.ones(dim)
    
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_std
        
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean 
                self.running_std  = (1 - self.momentum) * self.running_std + self.momentum * xvar 
        return self.out 
    
    def parameters(self):
        return [self.gamma,self.beta]
    
class Embedding:

    def __init__(self, num_embeddings, embedding_dim):
        self.weight = torch.randn((num_embeddings, embedding_dim))
        
    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out
    
    def parameters(self):
        return [self.weight]

class Flatten:

    def __call__(self, x):
        self.out = x.view(x.shape[0], -1)
        return self.out
    
    def parameters(self):
        return []
    
class Sequential:

    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self,):
        return [p for layer in self.layers for p in layer.parameters()]



In [19]:
torch.manual_seed(42);

In [20]:
vocab_size = 27
n_emb = 10
n_hidden = 200

In [23]:
model = Sequential([
    Embedding(vocab_size, n_emb),
    Flatten(),
    Linear(block_size * n_emb, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size)
])

# parameter init
with torch.no_grad():
    model.layers[-1].weight *= 0.1

In [24]:
print(sum(p.nelement() for p in model.parameters()))

12297


In [25]:
for p in model.parameters():
    p.requires_grad = True

In [27]:
max_steps = 200000
batch_size = 32
lossi = []

In [28]:
for k in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (32, ))
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward
    x = Xb
    logits = model(x)
    
    loss = F.cross_entropy(logits, Yb)
    lossi.append(loss)
    # backward pass
    for p in model.parameters():
        p.grad = None
    loss.backward()

    lr = 0.1 if k < 100000 else 0.01
    for p in model.parameters():
        p.data += -lr * p.grad 
    
    if k % 10000 == 0:
        print(f"{k:7d}/{max_steps:7d}: {loss.item(): .4f}")
    break


      0/ 200000:  27.3582


In [37]:
for layer in layers:
    layer.training = False

In [39]:
@torch.no_grad()
def split_loss(split):
    x,y  = {
        'train': (Xtr, Ytr),
        'valid': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, y)
    print(split, loss.item())

split_loss('train')
split_loss('valid')

train 2.2613141536712646
valid 2.2648017406463623
